In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import os.path

import base64
from email.message import EmailMessage

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]

load_dotenv(find_dotenv())
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [2]:
creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "credentials.json", SCOPES
        )
        creds = flow.run_local_server(port=4000)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())

service = None
try:
    # Call the Gmail API
    service = build("gmail", "v1", credentials=creds)

except HttpError as error:
# TODO(developer) - Handle errors from gmail API.
    print(f"An error occurred: {error}")

def getEmailList():
    try:
        results = service.users().messages().list(userId="me").execute()
        messages = results.get("messages", [])

        if not messages:
            print("No messages found.")
            return
        print("Message:")
        for message in messages:
            print(message)

    except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")

def gmail_send_message(content, to, receiver, subject):
  """Create and send an email message
  Returns: Message object, including message id

  Load pre-authorized user credentials from the environment.
  """

  try:
    message = EmailMessage()

    message.set_content(content)

    message["To"] = to
    message["From"] = receiver
    message["Subject"] = subject

    # encoded message
    encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

    create_message = {"raw": encoded_message}
    # pylint: disable=E1101
    send_message = (
        service.users()
        .messages()
        .send(userId="me", body=create_message)
        .execute()
    )
  except HttpError as error:
    print(f"An error occurred: {error}")
    send_message = None
  return send_message


if __name__ == "__main__":
  gmail_send_message()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=915927531429-jb5jkim9325sqr9bun1hvo1evtc961rq.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A4000%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=SZP9pKXL2aPLp96VmQDn1ItsNMSia8&access_type=offline


In [ ]:
# Creates a new assistant
assistant = client.beta.assistants.create(
    name = "Gmail Helper",
    instructions = "Your go-to assistant for reading and editing emails. Gmail Helper is a personal assistant dedicated to reading and sending emails, primarily through Google Gmail API integration. Its core functionality includes checking the user’s email to read and summarize emails they receive. It is capable of sending emails to the user's contacts but will always seek confirmation from the user before doing so, ensuring that the user has control over what they send. Gmail Helper maintains a professional demeanor, focusing on providing clear, concise, and accurate scheduling information. It prioritizes user privacy and confidentiality.The assistant is also equipped to guide users through the Google Gmail API’s functionalities and troubleshoot common issues, using language tailored to the user’s level of technical expertise. Overall, Gmail Helper adopts a polite, efficient, and accommodating tone, aiming to make email management and event scheduling as smooth and user-friendly as possible.",
    tools = [{
      "type": "function",
    "function": {
      "name": "getEmailList",
      "description": "Get a list of the user's mailbox contents",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
        },
        "required": ["location"]
      }
    }


  }
  
  {
      "type": "function",
    "function": {
      "name": "sendEmail",
      "description": "Send an email to another user",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
        },
        "required": ["location"]
      }
    }

    
  }],
    model = "gpt-4"
)

In [ ]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_dQORqEbykI5IOgf8KK2Nfm1D', created_at=1708713501, metadata={}, object='thread', tool_resources=[])


In [ ]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "What is my latest email?"
)

In [ ]:
print(message)

ThreadMessage(id='msg_9BjUTo8eeytUEcf81ZbhjRk3', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What is my latest email?'), type='text')], created_at=1708713501, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_dQORqEbykI5IOgf8KK2Nfm1D')


In [ ]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id
)

In [ ]:
run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)

In [ ]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)

In [ ]:
for message in reversed(messages.data):
    print(message.role + ': ' + message.content[0].text.value)

user: What is my latest email?


In [ ]:
service.close()